In [16]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Use your own account, password
DB_NAME = ""# dbname
DB_USER = ""# user
DB_PASSWORD = ""# password
DB_HOST = ""# host
DB_PORT = ""# port

# Fill in the path of the file
excel_path = "Survey_result_2025.xlsx" 

In [3]:
##load in data 

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

# Mapping between table names and Excel sheet names
table_sheet_info = {
    "students": "Section0",
    "values": "Section 1 - Value",
    "career_orientation": "Section2 - Career Orientation",
    "personality": "Section 3 - Personality",
    "creative_thinking": "Section 4 - creativity thinking",
    "family_info": "Section 5 - Family"
}

# Load Excel file
xlsx = pd.ExcelFile(excel_path)

# Loop through each table and insert data row by row
for table_name, sheet_name in table_sheet_info.items():
    df = xlsx.parse(sheet_name)
    records = df.values.tolist()
    num_fields = df.shape[1]
    placeholders = ', '.join(['%s'] * num_fields)
    insert_sql = f'INSERT INTO {table_name} VALUES ({placeholders})'

    cur.executemany(insert_sql, records)
    print(f"Successfully inserted into table {table_name}: {len(records)} records")

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

Successfully inserted into table students: 122 records
Successfully inserted into table values: 122 records
Successfully inserted into table career_orientation: 122 records
Successfully inserted into table personality: 122 records
Successfully inserted into table creative_thinking: 122 records
Successfully inserted into table family_info: 122 records


In [18]:
#data cleaning
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a connection to the PostgreSQL database
# Use your own account, password(postgresql://<username>:<password>@<host>:<port>/<dbname>)
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Define the table names (ensure these match the names in your database)
table_names = [
    "students",
    "values",
    "career_orientation",
    "personality",
    "creative_thinking",
    "family_info"
]

# Create two dictionaries to store the original data and the cleaned data
dfs_raw = {}      # Store the original DataFrame read from the database
dfs_cleaned = {}  # Store the copy that will be used for cleaning

# Read data from each table and store in dfs_raw; make a copy in dfs_cleaned
for table_name in table_names:
    try:
        # Read data from the database into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table_name}", engine)
        dfs_raw[table_name] = df
        print(f"Successfully loaded table '{table_name}', shape: {df.shape}")
        
        # Make a copy of the data for cleaning
        df_clean = df.copy()
        dfs_cleaned[table_name] = df_clean
    except Exception as e:
        print(f"Error loading table '{table_name}': {e}")

# Replace "na", "NA", and empty strings with actual missing values (pd.NA) in text columns of dfs_cleaned
for table_name, df in dfs_cleaned.items():
    object_cols = df.select_dtypes(include=["object"]).columns
    for col in object_cols:
        df[col] = df[col].replace(["na", "NA", ""], pd.NA)

# Preview the raw data stored in dfs_raw
sheets_preview = {sheet: df.head() for sheet, df in dfs_raw.items()}
for sheet, preview in sheets_preview.items():
    print(f"\nPreview of {sheet} (raw data):")
    print(preview)

Successfully loaded table 'students', shape: (122, 13)
Successfully loaded table 'values', shape: (122, 7)
Successfully loaded table 'career_orientation', shape: (122, 7)
Successfully loaded table 'personality', shape: (122, 10)
Successfully loaded table 'creative_thinking', shape: (122, 4)
Successfully loaded table 'family_info', shape: (122, 13)

Preview of students (raw data):
   identifier first_letter_surname gender  age first_language other_language  \
0           1                    L      M   14        English      Mandarine   
1           2                    Z      M   17       Mandarin        English   
2           3                    W      M   15       Mandarin        English   
3           4                    J      M   14        English       Mandarin   
4           5                    R      M   14       Mandarin        English   

  country_born current_country current_city type_of_school  \
0           UK       Australia       Sydney        Private   
1        Chi

In [20]:
# Check the data types in each sheet (using the cleaned data)
for sheet, df in dfs_cleaned.items():
    print(f"Data types in {sheet}:")
    print(df.dtypes)
    print("\n")

Data types in students:
identifier                  int64
first_letter_surname       object
gender                     object
age                         int64
first_language             object
other_language             object
country_born               object
current_country            object
current_city               object
type_of_school             object
secondary_qualification    object
work_experience            object
dream_career               object
dtype: object


Data types in values:
no                          int64
hedonism                    int64
power_and_status            int64
altruism                    int64
learning_and_achievement    int64
finance                     int64
security                    int64
dtype: object


Data types in career_orientation:
no               int64
artistic         int64
social           int64
investigative    int64
conventional     int64
enterprising     int64
realistic        int64
dtype: object


Data types in personality:
no  

In [22]:
# Check the missing values and duplicate rows in the cleaned data (dfs_cleaned)
for table, df in dfs_cleaned.items():
    print(f"Missing values in {table} (cleaned data):")
    print(df.isnull().sum())
    print(f"Duplicate rows in {table} (cleaned data): {df.duplicated().sum()}")
    print("\n")

Missing values in students (cleaned data):
identifier                 0
first_letter_surname       0
gender                     0
age                        0
first_language             0
other_language             0
country_born               0
current_country            0
current_city               0
type_of_school             0
secondary_qualification    0
work_experience            0
dream_career               0
dtype: int64
Duplicate rows in students (cleaned data): 0


Missing values in values (cleaned data):
no                          0
hedonism                    0
power_and_status            0
altruism                    0
learning_and_achievement    0
finance                     0
security                    0
dtype: int64
Duplicate rows in values (cleaned data): 0


Missing values in career_orientation (cleaned data):
no               0
artistic         0
social           0
investigative    0
conventional     0
enterprising     0
realistic        0
dtype: int64
Duplicate ro

In [24]:
# Process only the working copy (df_copy) without modifying the original (df)
for table, df in dfs_cleaned.items():
    # Get all object (text) columns
    object_cols = df.select_dtypes(include=["object"]).columns
    # Replace "", "na", "NA", "NaN", "nan" with actual missing value pd.NA
    for col in object_cols:
        df[col] = df[col].str.strip()  # Remove leading/trailing whitespace
        df[col] = df[col].replace(["", "na", "NA", "NaN", "nan"], pd.NA)
    # Print preview and missing value summary for verification
    print(f"[Processed] After replacement in table '{table}':")
    print(df.head(), "\n")
    print(f"Missing values in '{table}':")
    print(df.isnull().sum())
    print("\n")

[Processed] After replacement in table 'students':
   identifier first_letter_surname gender  age first_language other_language  \
0           1                    L      M   14        English      Mandarine   
1           2                    Z      M   17       Mandarin        English   
2           3                    W      M   15       Mandarin        English   
3           4                    J      M   14        English       Mandarin   
4           5                    R      M   14       Mandarin        English   

  country_born current_country current_city type_of_school  \
0           UK       Australia       Sydney        Private   
1        China       Australia       Sydney         Public   
2        China       Australia       Sydney        Private   
3        Japan              UK       Oxford        Private   
4        Japan           China       Harbin        Private   

  secondary_qualification work_experience     dream_career  
0                    ATAR         

In [26]:
# Check the missing values and duplicate rows in the cleaned data (dfs_cleaned)
for table, df in dfs_cleaned.items():
    print(f"Missing values in {table} (cleaned data):")
    print(df.isnull().sum())
    print(f"Duplicate rows in {table} (cleaned data): {df.duplicated().sum()}")
    print("\n")

Missing values in students (cleaned data):
identifier                  0
first_letter_surname        0
gender                      0
age                         0
first_language              0
other_language              0
country_born                0
current_country             0
current_city                0
type_of_school              0
secondary_qualification     0
work_experience            51
dream_career                9
dtype: int64
Duplicate rows in students (cleaned data): 0


Missing values in values (cleaned data):
no                          0
hedonism                    0
power_and_status            0
altruism                    0
learning_and_achievement    0
finance                     0
security                    0
dtype: int64
Duplicate rows in values (cleaned data): 0


Missing values in career_orientation (cleaned data):
no               0
artistic         0
social           0
investigative    0
conventional     0
enterprising     0
realistic        0
dtype: int64

In [28]:
# Get descriptive statistical information for the cleaned data
for sheet, df in dfs_cleaned.items():
    print(f"Descriptive statistics for {sheet}:")
    print(df.describe())
    print("\n")

Descriptive statistics for students:
       identifier         age
count  122.000000  122.000000
mean    61.500000   15.868852
std     35.362409    2.084777
min      1.000000   12.000000
25%     31.250000   15.000000
50%     61.500000   16.000000
75%     91.750000   18.000000
max    122.000000   19.000000


Descriptive statistics for values:
               no    hedonism  power_and_status    altruism  \
count  122.000000  122.000000        122.000000  122.000000   
mean    61.500000   28.655738         29.237705   27.852459   
std     35.362409   17.805084         17.369610   16.012739   
min      1.000000    0.000000          1.000000    1.000000   
25%     31.250000   13.250000         15.000000   13.000000   
50%     61.500000   28.000000         29.000000   25.500000   
75%     91.750000   44.750000         42.750000   40.750000   
max    122.000000   60.000000         59.000000   60.000000   

       learning_and_achievement     finance    security  
count                122.00000

In [30]:
# Check the column names of each sheet in the cleaned data
for sheet, df in dfs_cleaned.items():
    print(f"Columns in {sheet}:")
    print(df.columns)

Columns in students:
Index(['identifier', 'first_letter_surname', 'gender', 'age', 'first_language',
       'other_language', 'country_born', 'current_country', 'current_city',
       'type_of_school', 'secondary_qualification', 'work_experience',
       'dream_career'],
      dtype='object')
Columns in values:
Index(['no', 'hedonism', 'power_and_status', 'altruism',
       'learning_and_achievement', 'finance', 'security'],
      dtype='object')
Columns in career_orientation:
Index(['no', 'artistic', 'social', 'investigative', 'conventional',
       'enterprising', 'realistic'],
      dtype='object')
Columns in personality:
Index(['no', 'extrovert', 'introvert', 'sensing', 'intuition_n', 'thinking',
       'feeling', 'judging', 'perceiving', 'result_'],
      dtype='object')
Columns in creative_thinking:
Index(['no', 'q1', 'q2', 'q3'], dtype='object')
Columns in family_info:
Index(['no', 'father_education', 'mother_education', 'father_occupation',
       'mother_occupation', 'annual_b

In [32]:
# Unify the ID column names of all sheets in the cleaned data to 'Identifier'
for sheet, df in dfs_cleaned.items():
    if 'no' in df.columns:
        dfs_cleaned[sheet] = df.rename(columns={'no': 'identifier'})

In [34]:
# Check the column names of each sheet
for sheet, df in  dfs_cleaned.items():
    print(f"Columns in {sheet}:")
    print(df.columns)

Columns in students:
Index(['identifier', 'first_letter_surname', 'gender', 'age', 'first_language',
       'other_language', 'country_born', 'current_country', 'current_city',
       'type_of_school', 'secondary_qualification', 'work_experience',
       'dream_career'],
      dtype='object')
Columns in values:
Index(['identifier', 'hedonism', 'power_and_status', 'altruism',
       'learning_and_achievement', 'finance', 'security'],
      dtype='object')
Columns in career_orientation:
Index(['identifier', 'artistic', 'social', 'investigative', 'conventional',
       'enterprising', 'realistic'],
      dtype='object')
Columns in personality:
Index(['identifier', 'extrovert', 'introvert', 'sensing', 'intuition_n',
       'thinking', 'feeling', 'judging', 'perceiving', 'result_'],
      dtype='object')
Columns in creative_thinking:
Index(['identifier', 'q1', 'q2', 'q3'], dtype='object')
Columns in family_info:
Index(['identifier', 'father_education', 'mother_education',
       'father_occ

In [36]:
# Use the "students" table as the initial DataFrame
merged_df = dfs_cleaned["students"]

# Sequentially merge the other 5 tables
for table in ["values", "career_orientation", "personality", "creative_thinking", "family_info"]:
    merged_df = pd.merge(merged_df, dfs_cleaned[table], on="identifier", how="left")

# Preview the merged data
print("Preview of merged data:")
print(merged_df.head())

Preview of merged data:
   identifier first_letter_surname gender  age first_language other_language  \
0           1                    L      M   14        English      Mandarine   
1           2                    Z      M   17       Mandarin        English   
2           3                    W      M   15       Mandarin        English   
3           4                    J      M   14        English       Mandarin   
4           5                    R      M   14       Mandarin        English   

  country_born current_country current_city type_of_school  ...  \
0           UK       Australia       Sydney        Private  ...   
1        China       Australia       Sydney         Public  ...   
2        China       Australia       Sydney        Private  ...   
3        Japan              UK       Oxford        Private  ...   
4        Japan           China       Harbin        Private  ...   

         father_occupation              mother_occupation annual_budget_usd  \
0            

In [38]:
# Check the number of missing values for each column
missing_values = merged_df.isnull().sum()

# Print the columns that have missing values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

Columns with missing values:
work_experience       51
dream_career           9
result_               19
father_education       1
father_occupation      2
mother_occupation      3
preferred_foe_1        3
preferred_foe_2       20
preferred_foe_3       41
notes                118
concern               37
dtype: int64


In [40]:
# Check the number of duplicate rows
duplicate_rows = merged_df.duplicated().sum()

# Remove duplicate rows
merged_df = merged_df.drop_duplicates()

# Print information after removing duplicate rows
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


In [42]:
# Standardize language names
merged_df['first_language'] = merged_df['first_language'].replace({
    'Mandarin': 'Chinese',
    'China': 'Chinese',  # Handle the incorrect writing.
    'English': 'English',
    'Spanish': 'Spanish',
    'French': 'French'
})

merged_df['other_language'] = merged_df['other_language'].replace({
    'Mandarin': 'Chinese',
    'China': 'Chinese',  # Handle the incorrect writing.
    'English': 'English',
    'Spanish': 'Spanish',
    'French': 'French'
})

# Display the modified results
print(merged_df[['first_language', 'other_language']].head())

  first_language other_language
0        English      Mandarine
1        Chinese        English
2        Chinese        English
3        English        Chinese
4        Chinese        English


In [44]:
merged_df['secondary_qualification'] = merged_df['secondary_qualification'].replace({
    'A-level': 'A Level',
    'A Level': 'A Level',  # uniform case
    'IB': 'International Baccalaureate',
    'ATAR': 'Australian Tertiary Admission Rank',
    'Diploma': 'Diploma',
    'PhD': 'Doctorate',
    'Master\'s': 'Master\'s Degree',
    'Bachelor': 'Bachelor\'s Degree'
})

# Display the updated 'Secondary Qualification' column
print(merged_df['secondary_qualification'].head())

0    Australian Tertiary Admission Rank
1    Australian Tertiary Admission Rank
2           International Baccalaureate
3                               A Level
4                               A Level
Name: secondary_qualification, dtype: object


In [46]:
# Remove leading and trailing spaces from all target columns (column names are standardized to lowercase)
merged_df['first_language'] = merged_df['first_language'].str.strip()
merged_df['other_language'] = merged_df['other_language'].str.strip()
merged_df['secondary_qualification'] = merged_df['secondary_qualification'].str.strip()
merged_df['country_born'] = merged_df['country_born'].str.strip()
merged_df['current_country'] = merged_df['current_country'].str.strip()
merged_df['gender'] = merged_df['gender'].str.strip()  # Also clean up gender column

# -------------------------------
# Standardize gender representation
merged_df['gender'] = merged_df['gender'].replace({
    'M': 'Male',
    'F': 'Female'
})

# Display gender distribution
print(merged_df[['gender']].value_counts())

gender
Male      63
Female    59
Name: count, dtype: int64


In [48]:
# Standardize country names
merged_df['country_born'] = merged_df['country_born'].replace({
    'China': 'China',
    'UK': 'United Kingdom',
    'USA': 'United States',
    'Australia': 'Australia'
})

# Display the results
print(merged_df[['gender', 'country_born', 'current_country']].head())

  gender    country_born current_country
0   Male  United Kingdom       Australia
1   Male           China       Australia
2   Male           China       Australia
3   Male           Japan              UK
4   Male           Japan           China


In [50]:
print(merged_df.columns)

Index(['identifier', 'first_letter_surname', 'gender', 'age', 'first_language',
       'other_language', 'country_born', 'current_country', 'current_city',
       'type_of_school', 'secondary_qualification', 'work_experience',
       'dream_career', 'hedonism', 'power_and_status', 'altruism',
       'learning_and_achievement', 'finance', 'security', 'artistic', 'social',
       'investigative', 'conventional', 'enterprising', 'realistic',
       'extrovert', 'introvert', 'sensing', 'intuition_n', 'thinking',
       'feeling', 'judging', 'perceiving', 'result_', 'q1', 'q2', 'q3',
       'father_education', 'mother_education', 'father_occupation',
       'mother_occupation', 'annual_budget_usd', 'preferred_foe_1',
       'preferred_foe_2', 'preferred_foe_3', 'notes', 'concern', 'support',
       'preferred_country'],
      dtype='object')


In [52]:
# Rename column: change 'annual_budget_usd' to 'annual_budget_usd (usd)'
merged_df.rename(columns={'annual_budget_usd': 'annual_budget_usd (usd)'}, inplace=True)

# Remove "$" and "," symbols, then convert to numeric
merged_df['annual_budget_usd (usd)'] = merged_df['annual_budget_usd (usd)'].replace({r'\$': '', ',': ''}, regex=True)
merged_df['annual_budget_usd (usd)'] = pd.to_numeric(merged_df['annual_budget_usd (usd)'], errors='coerce')

# Display the results
print(merged_df[['annual_budget_usd (usd)']].head())

   annual_budget_usd (usd)
0                  80000.0
1                      NaN
2                      NaN
3                  30000.0
4                  50000.0


In [54]:

# Fill missing values (fill NaNs in 'annual_budget_usd (usd)' with the column mean)
merged_df['annual_budget_usd (usd)'] = merged_df['annual_budget_usd (usd)'].fillna(merged_df['annual_budget_usd (usd)'].mean())

# Drop rows where 'secondary_qualification' is missing
merged_df = merged_df.dropna(subset=['secondary_qualification'])

# Print remaining missing value counts
print(merged_df.isnull().sum())


identifier                    0
first_letter_surname          0
gender                        0
age                           0
first_language                0
other_language                0
country_born                  0
current_country               0
current_city                  0
type_of_school                0
secondary_qualification       0
work_experience              51
dream_career                  9
hedonism                      0
power_and_status              0
altruism                      0
learning_and_achievement      0
finance                       0
security                      0
artistic                      0
social                        0
investigative                 0
conventional                  0
enterprising                  0
realistic                     0
extrovert                     0
introvert                     0
sensing                       0
intuition_n                   0
thinking                      0
feeling                       0
judging 

In [56]:
# Define MBTI calculation function based on columns from the personality table
def calculate_mbti(row):
    # Compare extrovert vs introvert
    dim1 = 'E' if row['extrovert'] >= row['introvert'] else 'I'
    # Compare sensing vs intuition_n
    dim2 = 'S' if row['sensing'] >= row['intuition_n'] else 'N'
    # Compare thinking vs feeling
    dim3 = 'T' if row['thinking'] >= row['feeling'] else 'F'
    # Compare judging vs perceiving
    dim4 = 'J' if row['judging'] >= row['perceiving'] else 'P'
    
    return dim1 + dim2 + dim3 + dim4

# Fill missing values in the 'result_' column using the MBTI calculation based on personality traits
mask = merged_df['result_'].isnull()
merged_df.loc[mask, 'result_'] = merged_df.loc[mask].apply(calculate_mbti, axis=1)

# View the last 10 rows to verify MBTI calculation
print(merged_df[['extrovert','introvert','sensing','intuition_n','thinking','feeling','judging','perceiving','result_']].tail(10))

     extrovert  introvert  sensing  intuition_n  thinking  feeling  judging  \
112          0          3        0            3         0        3        0   
113          0          3        0            3         3        0        3   
114          0          3        0            3         3        0        0   
115          0          2        3            0         0        3        3   
116          0          3        2            0         0        3        0   
117          0          3        3            0         3        0        3   
118          0          3        3            0         3        0        0   
119          0          2        0            0         1        0        0   
120          0          4        3            0         0        2        1   
121          0          1        3            0         0        1        1   

     perceiving result_  
112           3    INFP  
113           0    INTJ  
114           3    INTP  
115           0    ISFJ  


In [58]:
# Replace the missing values of 'concern' with 'Other'
merged_df['concern'] = merged_df['concern'].fillna('Other')
print(merged_df['concern'].unique())

['Yes' 'No' 'Other' 'yes']


In [60]:
merged_df['concern'] = merged_df['concern'].replace('yes', 'Yes')
print(merged_df['concern'].unique())

['Yes' 'No' 'Other']


In [62]:
# Save the cleaned data to a CSV file
merged_df.to_csv('cleaned_full_data.csv', index=False)

# Or save it as an Excel file
merged_df.to_excel('cleaned_full_data.xlsx', index=False)

In [64]:
 #Save final cleaned and merged data to the PostgreSQL database as a new table
merged_df.to_sql(
    name='cleaned_full_data',   # This creates a new table
    con=engine,
    if_exists='replace',        # Replace only if this table already exists (safe!)
    index=False
)

print("cleaned_full_data' saved successfully without affecting original tables.")

cleaned_full_data' saved successfully without affecting original tables.
